# Tutorial 3: Seasonality

We will explore the seasonality component of NeuralProphet. It allows to model to capture seasonal effects in the data, for example effects which repeat the same time every year or the same day of every week.

We start with the same code as in the previous tutorial on trends.

In [1]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

# Disable logging messages unless there is an error
set_log_level("ERROR")

# Load the dataset from the CSV file using pandas
df = pd.read_csv("https://github.com/ourownstory/neuralprophet-data/raw/main/kaggle-energy/datasets/tutorial01.csv")

# Reduce the number of epochs to 5 for faster training
EPOCHS = 5

# Model and prediction
m = NeuralProphet(
    epochs=EPOCHS,
    # Disable trend changepoints
    n_changepoints=0,
    # Disable seasonality components
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False,
)
metrics = m.fit(df)
forecast = m.predict(df)
m.plot(forecast)

Finding best initial lr:   0%|          | 0/229 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 46it [00:00, ?it/s]

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': 'rgba(45, 146, 255, 1.0)', 'width': 2},
              'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> yhat1 <i style="color:#fc9944">~1D</i>',
              'type': 'scatter',
              'uid': '05719373-e8f6-4ae7-8eb1-824d071c5000',
              'x': array([datetime.datetime(2014, 12, 31, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0), ...,
                          datetime.datetime(2018, 12, 28, 0, 0),
                          datetime.datetime(2018, 12, 29, 0, 0),
                          datetime.datetime(2018, 12, 31, 0, 0)], dtype=object),
              'y': array([63.74633026, 63.74136353, 63.73639679, ..., 56.50798798, 56.50302887,
                          56.4930954 ])},
             {'marker': {'color': 'black', 'size': 4},
              'mode': 'markers',
              'n

Let us enable the seasonality step by step again, starting with the yearly seasonality. We give it a try and then see what effects it has on the model.

In [2]:
# Model and prediction with yearly seasonality
m = NeuralProphet(
    epochs=EPOCHS,
    # Disable trend changepoints
    n_changepoints=0,
    # Disable seasonality components, except yearly
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
)
metrics = m.fit(df)
forecast = m.predict(df)
m.plot(forecast)

Finding best initial lr:   0%|          | 0/229 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 46it [00:00, ?it/s]

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': 'rgba(45, 146, 255, 1.0)', 'width': 2},
              'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> yhat1 <i style="color:#fc9944">~1D</i>',
              'type': 'scatter',
              'uid': 'a261864d-75cc-4bbf-8f61-d1aab1e7a887',
              'x': array([datetime.datetime(2014, 12, 31, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0), ...,
                          datetime.datetime(2018, 12, 28, 0, 0),
                          datetime.datetime(2018, 12, 29, 0, 0),
                          datetime.datetime(2018, 12, 31, 0, 0)], dtype=object),
              'y': array([49.57288361, 49.68101501, 49.79979706, ..., 71.75550079, 71.82727814,
                          72.00841522])},
             {'marker': {'color': 'black', 'size': 4},
              'mode': 'markers',
              'n

Plotting all components of the forecast we see that the forecast is composed of the trend and the newly added seasonality in an additive way. This means our prediction $\hat{y}$ is the sum of the trend and the seasonality:

$$\text{Prediction}(t) = \hat{y}(t) = \text{Trend}(t) + \text{Seasonality}_{yearly}(t)$$

We can see this more cleary when we plot the trend and the seasonality components separately.

In [3]:
m.plot_components(forecast)

FigureWidgetResampler({
    'data': [{'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> Trend <i style="color:#fc9944">~1D</i>',
              'showlegend': False,
              'type': 'scatter',
              'uid': '630b1a31-47a2-44f6-86be-c165fa5ae8db',
              'x': array([datetime.datetime(2014, 12, 31, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0), ...,
                          datetime.datetime(2018, 12, 28, 0, 0),
                          datetime.datetime(2018, 12, 29, 0, 0),
                          datetime.datetime(2018, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([45.07519913, 45.09055328, 45.10591125, ..., 67.46466827, 67.48001862,
                          67.51073456]),
              'yaxis': 'y'},
             {'line': {'color': '#2d92ff', 'width': 2},
   

An alternative way is to plot the forecast parameters directly. In this visualization we see the contribution of each components independent of the whole dataset, meaning only a single year for the yearly seasonality.

The functions `plot_paramters` and `plot_components` allow you to specify which components to plot. So for the next visualizations we will focus on the seasonality alone (respectively not showing the trend).

In [4]:
m.plot_parameters(components=["seasonality"])

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': 'yearly',
              'type': 'scatter',
              'uid': '247554f5-b9a3-4ffe-84e6-d73bb7c8f347',
              'x': array([datetime.datetime(2017, 1, 1, 0, 0),
                          datetime.datetime(2017, 1, 2, 0, 0),
                          datetime.datetime(2017, 1, 3, 0, 0), ...,
                          datetime.datetime(2017, 12, 29, 0, 0),
                          datetime.datetime(2017, 12, 30, 0, 0),
                          datetime.datetime(2017, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([4.64090167, 4.74928054, 4.86675368, ..., 4.36336949, 4.43559245,
                          4.51983006]),
              'yaxis': 'y'}],
    'layout': {'autosize': True,
               'font': {'size': 10},
               'height': 210,
               'hovermode': 'x unified'

Seasonality in NeuralProphet is handled by using Fourier terms. You can think of Fourier terms as the sum of one or many overlayed sines and cosines functions. This way NeuralProphet can model multiple seasonalities. And the seasonality can be of arbitrary periodicities (length), for example a yearly seasonality with daily data ($p = 365.25$) or with weekly data ($p = 52.18$).

You can read more on [Fourier terms on Wikipedia](https://en.wikipedia.org/wiki/Fourier_series) and about the detailed inner workings of seasonality in the [paper on NeuralProphet](https://arxiv.org/abs/2001.04063) under section `2.1.2 Seasonality`.

We continue by enabling the `weekly` and `daily` seasonality.

In [5]:
# Model and prediction
m = NeuralProphet(
    epochs=EPOCHS,
    # Disable trend changepoints
    n_changepoints=0,
    # Enable all seasonality components (default for NeuralProphet)
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
)
metrics = m.fit(df)
forecast = m.predict(df)
m.plot(forecast)

Finding best initial lr:   0%|          | 0/229 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 46it [00:00, ?it/s]

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': 'rgba(45, 146, 255, 1.0)', 'width': 2},
              'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> yhat1 <i style="color:#fc9944">~1D</i>',
              'type': 'scatter',
              'uid': '6b320975-416e-4300-b19e-cab7476bb7e7',
              'x': array([datetime.datetime(2014, 12, 31, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0), ...,
                          datetime.datetime(2018, 12, 28, 0, 0),
                          datetime.datetime(2018, 12, 30, 0, 0),
                          datetime.datetime(2018, 12, 31, 0, 0)], dtype=object),
              'y': array([59.14366913, 57.30839539, 57.30134583, ..., 67.76183319, 59.26858521,
                          67.28587341])},
             {'marker': {'color': 'black', 'size': 4},
              'mode': 'markers',
              'n

In [6]:
m.plot_components(forecast, components=["seasonality"])

FigureWidgetResampler({
    'data': [{'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... ' style="color:#fc9944">~1D</i>'),
              'showlegend': False,
              'type': 'scatter',
              'uid': '29112611-adc7-457d-9980-d66c71d5bfd2',
              'x': array([datetime.datetime(2014, 12, 31, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0), ...,
                          datetime.datetime(2018, 12, 28, 0, 0),
                          datetime.datetime(2018, 12, 30, 0, 0),
                          datetime.datetime(2018, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([3.04829502, 2.94358516, 2.8548975 , ..., 3.45520663, 3.168782  ,
                          3.04829502]),
              'yaxis': 'y'},
             {'line': {'color': '#2d92ff', 'width': 2},
              

Daily seasonality does not make sense for this dataset. We see this clearly in the plotted as it only shows a flat line. For the weekly seasonality best zoom in or check out the plotted paramters below.

In [7]:
m.plot_parameters(components=["seasonality"])

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': 'yearly',
              'type': 'scatter',
              'uid': 'a9cbf55e-71d6-47cf-ad04-0e8742fd89f8',
              'x': array([datetime.datetime(2017, 1, 1, 0, 0),
                          datetime.datetime(2017, 1, 2, 0, 0),
                          datetime.datetime(2017, 1, 3, 0, 0), ...,
                          datetime.datetime(2017, 12, 29, 0, 0),
                          datetime.datetime(2017, 12, 30, 0, 0),
                          datetime.datetime(2017, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([2.89723074, 2.81658759, 2.75198828, ..., 3.26925144, 3.13720132,
                          3.020626  ]),
              'yaxis': 'y'},
             {'fill': 'none',
              'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': '